# 文本情感分类：使用循环神经网络

文本分类是自然语言处理的一个常见任务，它把一段不定长的文本序列变换为文本的类别。本节关注它的一个子问题：使用文本情感分类来分析文本作者的情绪。这个问题也叫情感分析，并有着广泛的应用。例如，我们可以分析用户对产品的评论并统计用户的满意度，或者分析用户对市场行情的情绪并用以预测接下来的行情。

同搜索近义词和类比词一样，文本分类也属于词嵌入的下游应用。在本节中，我们将应用预训练的词向量和含多个隐藏层的双向循环神经网络，来判断一段不定长的文本序列中包含的是正面还是负面的情绪。

在实验开始前，导入所需的包或模块。

In [0]:
import collections
import d2ltorch as d2lt
import torch
from torch import nn, optim
from torch.utils import data as tdata
import torchtext as text
import os
import random
import tarfile

## 文本情感分类数据

我们使用斯坦福的IMDb数据集（Stanford's Large Movie Review Dataset）作为文本情感分类的数据集 [1]。这个数据集分为训练和测试用的两个数据集，分别包含25,000条从IMDb下载的关于电影的评论。在每个数据集中，标签为“正面”和“负面”的评论数量相等。

###  读取数据

首先下载这个数据集到`../data`路径下，然后解压至`../data/aclImdb`下。


*注：可以使用 [`torchtext.datasets.IMDB(path, text_field, label_field, **kwargs)`](https://torchtext.readthedocs.io/en/latest/datasets.html#torchtext.datasets.IMDB) 直接获取数据集对象，这里为了方便后面的实验，还是采用下载的方式获取数据集*

In [4]:
# 本函数已保存在d2ltorch包中方便以后使用
def download_imdb(path='../data/sentiment/', fname='aclImdb_v1.tar.gz'):
    if not os.path.exists(path):
        os.mkdir(path)
    if not os.path.isfile(path + fname):
        url = ('http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz')
        text.utils.download_from_url(url, path+fname)
    with tarfile.open(path+fname, 'r') as f:
        f.extractall(path)

download_imdb()

aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:02<00:00, 38.4MB/s]


接下来，读取训练数据集和测试数据集。每个样本是一条评论及其对应的标签：1表示“正面”，0表示“负面”。

In [0]:
def read_imdb(folder='train'):  # 本函数已保存在d2ltorch包中方便以后使用
    data = []
    for label in ['pos', 'neg']:
        folder_name = os.path.join('../data/sentiment/aclImdb/', folder, label)
        for file in os.listdir(folder_name):
            with open(os.path.join(folder_name, file), 'rb') as f:
                review = f.read().decode('utf-8').replace('\n', '').lower()
                data.append([review, 1 if label == 'pos' else 0])
    random.shuffle(data)
    return data

train_data, test_data = read_imdb('train'), read_imdb('test')

### 预处理数据

我们需要对每条评论做分词，从而得到分好词的评论。这里定义的`get_tokenized_imdb`函数使用最简单的方法：基于空格进行分词。

In [0]:
def get_tokenized_imdb(data):  # 本函数已保存在d2ltorch包中方便以后使用
    def tokenizer(text):
        return [tok.lower() for tok in text.split(' ')]
    return [tokenizer(review) for review, _ in data]

现在，我们可以根据分好词的训练数据集来创建词典了。我们在这里过滤掉了出现次数少于5的词。

In [7]:
def get_vocab_imdb(data):  # 本函数已保存在d2ltorch包中方便以后使用
    tokenized_data = get_tokenized_imdb(data)
    counter = collections.Counter([tk for st in tokenized_data for tk in st])
    return text.vocab.Vocab(counter, min_freq=5)

vocab = get_vocab_imdb(train_data)
'# words in vocab:', len(vocab)

('# words in vocab:', 46151)

因为每条评论长度不一致所以不能直接组合成小批量，我们定义`preprocess_imdb`函数对每条评论进行分词，并通过词典转换成词索引，然后通过截断或者补0来将每条评论长度固定成500。

In [0]:
def preprocess_imdb(data, vocab):  # 本函数已保存在d2ltorch包中方便以后使用
    max_l = 500  # 将每条评论通过截断或者补0，使得长度变成500

    def pad(x):
        return x[:max_l] if len(x) > max_l else x + [0] * (max_l - len(x))

    tokenized_data = get_tokenized_imdb(data)
    indices = []
    for review in tokenized_data:
        indices.append(pad([vocab.stoi[w] for w in review]))
    features = torch.tensor(indices)
    labels = torch.tensor([score for _, score in data])
    return features, labels

### 创建数据迭代器

现在，我们创建数据迭代器。每次迭代将返回一个小批量的数据。

In [0]:
batch_size = 64
train_set = tdata.TensorDataset(*preprocess_imdb(train_data, vocab))
test_set = tdata.TensorDataset(*preprocess_imdb(test_data, vocab))
train_iter = tdata.DataLoader(train_set, batch_size, shuffle=True)
test_iter = tdata.DataLoader(test_set, batch_size)

打印第一个小批量数据的形状以及训练集中小批量的个数。

In [10]:
for X, y in train_iter:
    print('X', X.shape, 'y', y.shape)
    break
'#batches:', len(train_iter)

X torch.Size([64, 500]) y torch.Size([64])


('#batches:', 391)

## 使用循环神经网络的模型

首先加载预训练的词向量。由于情感分类的训练数据集并不是很大，为应对过拟合，我们将直接使用在更大规模语料上预训练的词向量作为每个词的特征向量。这里，我们为词典`vocab`中的每个词加载100维的GloVe词向量。

In [11]:
vocab.load_vectors(text.vocab.GloVe(name='6B', dim=100))

.vector_cache/glove.6B.zip: 862MB [00:16, 52.3MB/s]                           
100%|█████████▉| 398480/400000 [00:17<00:00, 23644.05it/s]

### 定义模型
在这个模型中，每个词先通过嵌入层得到特征向量。然后，我们使用双向循环神经网络对特征序列进一步编码得到序列信息。最后，我们将编码的序列信息通过全连接层变换为输出。具体来说，我们可以将双向长短期记忆在最初时间步和最终时间步的隐藏状态连结，作为特征序列的表征传递给输出层分类。在下面实现的`BiRNN`类中，`Embedding`实例即嵌入层，`LSTM`实例即为序列编码的隐藏层，`Dense`实例即生成分类结果的输出层。

In [0]:
class BiRNN(nn.Module):
    def __init__(self, vocab, embed_size, num_hiddens, num_layers, **kwargs):
        super(BiRNN, self).__init__(**kwargs)
        self.embedding = nn.Embedding(len(vocab), embed_size).from_pretrained(vocab.vectors)
        # bidirectional设为True即得到双向循环神经网络
        self.encoder = nn.LSTM(input_size=embed_size, hidden_size=num_hiddens, num_layers=num_layers,
                                bidirectional=True)
        self.decoder = nn.Linear(4 * num_hiddens, 2)

    def forward(self, inputs):
        # inputs的形状是(批量大小, 词数)，因为LSTM需要将序列作为第一维，所以将输入转置后
        # 再提取词特征，输出形状为(词数, 批量大小, 词向量维度)
        embeddings = self.embedding(inputs.t())
        # rnn.LSTM只传入输入embeddings，因此只返回最后一层的隐藏层在各时间步的隐藏状态。
        # outputs形状是(词数, 批量大小, 2 * 隐藏单元个数)
        outputs, hidden = self.encoder(embeddings)
        # 连结初始时间步和最终时间步的隐藏状态作为全连接层输入。它的形状为
        # (批量大小, 4 * 隐藏单元个数)。
        encoding = torch.cat([outputs[0], outputs[-1]], dim=1)
        outs = self.decoder(encoding)
        return outs

创建一个含两个隐藏层的双向循环神经网络。

In [0]:
embed_size, num_hiddens, num_layers = 100, 100, 2
flag, device = d2lt.try_all_gpus()
net = BiRNN(vocab, embed_size, num_hiddens, num_layers)
d2lt.params_init(net, nn.init.xavier_uniform_)

### 训练并评价模型

这时候就可以开始训练模型了。

In [54]:
lr, num_epochs = 0.01, 5
optimizer = optim.Adam(net.parameters(), lr=lr)
loss = nn.CrossEntropyLoss()
train(train_iter, test_iter, net, loss, optimizer, flag, device, num_epochs)

training on cuda
epoch 1, loss 0.5587, train acc 0.703, test acc 0.774, time 43.6 sec
epoch 2, loss 0.3995, train acc 0.820, test acc 0.827, time 44.6 sec
epoch 3, loss 0.3571, train acc 0.846, test acc 0.831, time 45.1 sec
epoch 4, loss 0.3240, train acc 0.861, test acc 0.827, time 45.3 sec
epoch 5, loss 0.2996, train acc 0.873, test acc 0.850, time 44.2 sec


最后，定义预测函数。

In [0]:
# 本函数已保存在d2ltorch包中方便以后使用
def predict_sentiment(net, vocab, sentence):
    sentence = torch.tensor([vocab.stoi[w] for w in sentence], device=d2lt.try_gpu())
    with torch.no_grad():
        label = torch.argmax(net(sentence.reshape(1, -1)), dim=1)
    return 'positive' if label.item() == 1 else 'negative'

下面使用训练好的模型对两个简单句子的情感进行分类。

In [57]:
predict_sentiment(net, vocab, ['this', 'movie', 'is', 'so', 'great'])

'positive'

In [58]:
predict_sentiment(net, vocab, ['this', 'movie', 'is', 'so', 'bad'])

'negative'

## 小结

* 文本分类把一段不定长的文本序列变换为文本的类别。它属于词嵌入的下游应用。
* 可以应用预训练的词向量和循环神经网络对文本的情感进行分类。


## 练习

* 增加迭代周期。训练后的模型能在训练和测试数据集上得到怎样的准确率？再调节其他超参数试试？

* 使用更大的预训练词向量，如300维的GloVe词向量，能否提升分类准确率？

* 使用spaCy分词工具，能否提升分类准确率？你需要安装spaCy（`pip install spacy`），并且安装英文包（`python -m spacy download en`）。在代码中，先导入spacy（`import spacy`）。然后加载spacy英文包（`spacy_en = spacy.load('en')`）。最后定义函数`def tokenizer(text): return [tok.text for tok in spacy_en.tokenizer(text)]`并替换原来的基于空格分词的`tokenizer`函数。需要注意的是，GloVe词向量对于名词词组的存储方式是用“-”连接各个单词，例如，词组“new york”在GloVe词向量中的表示为“new-york”，而使用spaCy分词之后“new york”的存储可能是“new york”。






## 参考文献

[1] Maas, A. L., Daly, R. E., Pham, P. T., Huang, D., Ng, A. Y., & Potts, C. (2011, June). Learning word vectors for sentiment analysis. In Proceedings of the 49th annual meeting of the association for computational linguistics: Human language technologies-volume 1 (pp. 142-150). Association for Computational Linguistics.

## 扫码直达[讨论区](https://discuss.gluon.ai/t/topic/6155)

![](../img/qr_sentiment-analysis.svg)